# Style Time Evaluation.
This script will evaluate Style Time As the Paper do.

### Import and Functions

In [ ]:
import os 
os.environ["VISIBLE_DEVICES"] = '-1'

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
def lstm_model(seq_shape:tuple) -> tf.keras.Model:
    """Make the LSTM Model as stated in the paper (2x100 lstm layers.)

    Args:
        seq_shape (tuple): The Shape of the sequence.

    Returns:
        tf.keras.Model: The model.
    """
    
    model = tf.keras.Sequential()
    print(seq_shape)
    model.add(tf.keras.layers.Input(seq_shape))
    model.add(tf.keras.layers.Reshape((seq_shape[-1], seq_shape[-2])))
    
    model.add(tf.keras.layers.LSTM(100, return_sequences=True))
    model.add(tf.keras.layers.LSTM(100, return_sequences=True))
    
    model.add(
        tf.keras.layers.TimeDistributed(
            tf.keras.layers.Dense(seq_shape[0])
        )
    )
    
    model.add(tf.keras.layers.Reshape(seq_shape))
    
    model.compile(
        optimizer='Adam', 
        loss="mae", 
        metrics=tf.keras.metrics.MeanAbsoluteError()
    )
    
    return model



In [ ]:
def plot_learning_curves(hist):
    plt.figure(figsize=(10, 5))
    
    plt.plot(hist.history['loss'], '.-', label=" Train Loss")
    plt.plot(hist.history['val_loss'], '.-', label="Valid Loss")
    
    plt.grid()
    plt.show()

def train_model(train_dset:np.ndarray, valid_dset:np.ndarray, epochs=50) -> tf.keras.Model:
    model = lstm_model(train_dset.shape[1:])
    
    hist = model.fit(train_dset, train_dset, validation_data=(valid_dset, valid_dset), epochs=epochs)
    
    plot_learning_curves(hist)
    
    return model

In [ ]:
def evaluate_model(model:tf.keras.Model, test_dataset:np.ndarray)-> float:
    reconstructions = model(test_dataset)
    
    return np.mean((np.sum(np.abs(reconstructions - test_dataset), axis=-1)))

## GOOGLE STOCKS RESULTS.
### TSTR MAE on "In Sample" Dataset.

In this part, we train the model on the "style transfered" data. Then, we evaluate it on the real data.

In [ ]:
in_sample_train = np.load("data/google_stocks/preprocessed/style_train.npy")
in_sample_test = np.load("data/google_stocks/preprocessed/style_test.npy")

transfered_in_samples_train = np.load("data/google_stocks/preprocessed/normal_content_train_transfered.npy")
transfered_in_samples_test = np.load("data/google_stocks/preprocessed/normal_content_test_transfered.npy")

# in_sample_trained_model = train_model(transfered_in_samples_train, in_sample_test, epochs=100)
in_sample_trained_model = train_model(transfered_in_samples_train, transfered_in_samples_test, epochs=100)

#### plot some sequences

In [ ]:
def plot_reconstruction(model, sequence, title=""):
    model_reconstruction = model(np.array([sequence]))[0]
    plt.figure(figsize=(10, 5))

    ax = plt.subplot(111)
    ax.set_title(title)
    ax.plot(sequence[0], ".-", label='model True')
    ax.grid(True)

    ax.plot(model_reconstruction[0], ".-", label='model Reconstruction')

    ax.grid(True)
    ax.legend()
    plt.show()
    
plot_reconstruction(
    in_sample_trained_model, 
    in_sample_test[0],
    "Comparison between the model reconstruction and the ground truth.\n Here on a $REAL$ $SEQUENCE$")

In [ ]:
plot_reconstruction(
    in_sample_trained_model, 
    transfered_in_samples_test[0],
    "Comparison between the model reconstruction and the ground truth.\n Here on a $FAKE$ $SEQUENCE$")

#### Evaluate the model 
With the Metric paper's metric: MAE

In [ ]:
in_sample_model_real_mae = evaluate_model(in_sample_trained_model, in_sample_test)
in_sample_model_fake_mae = evaluate_model(in_sample_trained_model, transfered_in_samples_test)

in_sample_model_real_mae, in_sample_model_fake_mae

### Test on "Perturbed" Dataset.

In [ ]:
perturbed_generated_train = np.load("data/google_stocks/preprocessed/perturbed_content_train_transfered.npy")
perturbed_generated_test = np.load("data/google_stocks/preprocessed/perturbed_content_test_transfered.npy")

perturbed_trained_model = train_model(perturbed_generated_train, perturbed_generated_test, epochs=100)

#### Plot Reconstructions

In [ ]:
plot_reconstruction(
    perturbed_trained_model, 
    in_sample_test[0],
    "Comparison between the model reconstruction and the ground truth.\n Here on a $REAL$ $SEQUENCE$")

plot_reconstruction(
    perturbed_trained_model, 
    perturbed_generated_test[1],
    "Comparison between the model reconstruction and the ground truth.\n Here on a $FAKE$ $SEQUENCE$")

#### Evaluate the model

In [ ]:
perturbed_model_real_mae = evaluate_model(perturbed_trained_model, in_sample_test)
perturbed_model_fake_mae = evaluate_model(perturbed_trained_model, perturbed_generated_test)

perturbed_model_real_mae, perturbed_model_fake_mae

#### Generate a DataFrame with the results.

In [ ]:
values = np.array([[in_sample_model_real_mae],  [perturbed_model_real_mae]])
# values = 1- values
cols = ["Google Stocks TSTR MAE"]
index = ["Style Time (In Samples)", "Style Time (Perturbed)"]

google_stocks_results = pd.DataFrame(index=index, columns=cols, data=values)

google_stocks_results

## Energy Dataset
### Again, TSTR MAE on "In Sample" Dataset

In [ ]:
energy_real_train = np.load("data/energy/preprocessed/style_train.npy")
energy_real_test = np.load("data/energy/preprocessed/style_test.npy")

energy_is_train = np.load("data/energy/preprocessed/in_sample_train.npy")
energy_is_test = np.load("data/energy/preprocessed/in_sample_train.npy")

energy_is_model = train_model(energy_is_train, energy_is_test, epochs=100)

In [ ]:
plot_reconstruction(
    energy_is_model, 
    energy_real_test[0],
    "Comparison between the model reconstruction and the ground truth.\n Here on a $REAL$ $SEQUENCE$"
    )

plot_reconstruction(
    energy_is_model, 
    energy_is_test[0],
    "Comparison between the model reconstruction and the ground truth.\n Here on a $FAKE$ $SEQUENCE$"
    )

### TSTR MAE on "perturbed" dataset.

In [ ]:
energy_pert_train = np.load("data/energy/preprocessed/perturbed_train.npy")
energy_pert_test = np.load("data/energy/preprocessed/perturbed_test.npy")

energy_pert_model = train_model(energy_pert_train, energy_pert_test, epochs=100)

In [ ]:
plot_reconstruction(
    energy_pert_model, 
    energy_real_test[0],
    "Comparison between the model reconstruction and the ground truth.\n Here on a $REAL$ $SEQUENCE$"
    )

plot_reconstruction(
    energy_pert_model, 
    energy_pert_test[0],
    "Comparison between the model reconstruction and the ground truth.\n Here on a $FAKE$ $SEQUENCE$"
    )

### Evaluate the model.

In [ ]:
energy_is_res_on_real = evaluate_model(energy_is_model, energy_real_test)
energy_is_res_on_fake = evaluate_model(energy_is_model, energy_is_test)

energy_is_res_on_real, energy_is_res_on_fake

In [ ]:
energy_pert_res_on_real = evaluate_model(energy_pert_model, energy_real_test)
energy_pert_res_on_fake = evaluate_model(energy_pert_model, energy_pert_test)

energy_pert_res_on_real, energy_pert_res_on_fake

### Make the DataFrame

In [ ]:
values = np.array([[energy_is_res_on_real],  [energy_pert_res_on_real]])
# values = 1- values
cols = ["Energy TSTR MAE"]
index = ["Style Time (In Samples)", "Style Time (Perturbed)"]

df_energy_results = pd.DataFrame(index=index, columns=cols, data=values)

df_energy_results

In [ ]:
google_stocks_results.merge(df_energy_results, left_index=True, right_index=True)